# Implicit Julia
* Converts the model created by the implicit package into Julia

In [1]:
name = "Implicit";
residual_alphas = [];

In [2]:
using DelimitedFiles
using GZip

In [3]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [4]:
BLAS.set_num_threads(Threads.nthreads())

In [5]:
item_factors = readdlm(GZip.open("../../data/alphas/Implicit/als_item_factors.csv.gz"), ',')
user_factors =
    collect(readdlm(GZip.open("../../data/alphas/Implicit/als_user_factors.csv.gz"), ',')');

In [6]:
user_factors =
    collect(readdlm(GZip.open("../../data/alphas/Implicit/als_user_factors.csv.gz"), ',')');

In [7]:
function pred(u, i)
    raw = item_factors * user_factors[:, u]
    clamp!(raw, 1e-20, Inf)
    raw = raw ./ sum(raw)
    raw[i]
end

pred (generic function with 1 method)

In [8]:
function make_prediction(users, items)
    p = zeros(length(users))
    @tprogress Threads.@threads for u in collect(Set(users))
        mask = users .== u
        p[mask] .= pred(u, items[mask])
    end
    p
end

make_prediction (generic function with 1 method)

In [11]:
write_predictions(make_prediction, residual_alphas = [], outdir = name, implicit = true)

Progress: 100%|███████████████████████████| Time: 0:21:19 (18.73 ms/it)m
[ Info: 20220312 14:07:57 validation set: Cross-Entropy loss 7.287955894055533
Progress: 100%|███████████████████████████| Time: 0:21:14 (18.66 ms/it)
